In [4]:
!pip install -U langchain-huggingface


In [1]:
!pip install pandas faiss-cpu sentence-transformers accelerate
!pip install -U langchain-huggingface langchain-community

In [1]:
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import faiss
import numpy as np
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import torch

In [3]:
# Load CSV file

csv_path = "hotel_bookings_cleaned.csv"  # Path to uploaded file
df = pd.read_csv(csv_path)

# Convert CSV data into text chunks
text_data = df.to_string(index=False)  # Convert entire CSV to string format

# Split the data into chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(text_data)


In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert text chunks into embeddings and store in FAISS
documents = [Document(page_content=chunk) for chunk in chunks]  # Wrap text in Document format
vector_store = FAISS.from_documents(documents, embedding_model)  # Create FAISS index

# Save FAISS index for later use (optional)
vector_store.save_local("faiss_index")



In [ ]:
import requests
import os

# 🔹 Hugging Face API Details
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN", "YOUR_HUGGINGFACE_TOKEN")  # Use env variable for security

HEADERS = {"Authorization": f"Bearer {HF_TOKEN}"}

def query_huggingface_api(prompt):
    """Sends a request to the Hugging Face API and handles errors."""
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 150,  # Lower token limit for efficiency
            "temperature": 0.7,  # Adds slight randomness for better results
            "top_p": 0.9  # Controls response diversity
        }
    }

    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload)
        response.raise_for_status()  # Raise error for bad responses
        result = response.json()

        # Extract generated text if available
        if isinstance(result, list) and "generated_text" in result[0]:
            generated_text = result[0]["generated_text"].strip()
            return generated_text if generated_text else "No meaningful response generated."
        return "Unexpected response format."

    except requests.exceptions.RequestException as e:
        return f"API Error: {str(e)}"

# 🔹 Run Query
query = "calculate which country has highest number of hotel bookings?"
retrieved_docs = retriever.get_relevant_documents(query)

if not retrieved_docs:
    response = "No relevant information found in the database."
else:
    # Use only 1 document and limit its length
    context = " ".join([doc.page_content for doc in retrieved_docs[:1]])[:1000]

    prompt = f"Context: {context}\nQuestion: {query}\nAnswer (only generate the final and precise solution,in one sentence):"

    response = query_huggingface_api(prompt)

print(response)


Context: hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  arrival_date_week_number  arrival_date_day_of_month  stays_in_weekend_nights  stays_in_week_nights  adults  children  babies      meal country market_segment distribution_channel  is_repeated_guest  previous_cancellations  previous_bookings_not_canceled reserved_room_type assigned_room_type  booking_changes deposit_type  agent  company  days_in_waiting_list   customer_type     adr  required_car_parking_spaces  total_of_special_requests reservation_status reservation_status_date
Resort Hotel            0        342               2015               July                        27                          1                        0                     0       2         0       0        BB     PRT         Direct               Direct                  0                       0                               0                  C                  C                3   No Deposit      0        0                     0    

In [29]:
# 🔹 Run Query 2
query = "What is the most common market segment for hotel bookings?"
retrieved_docs = retriever.get_relevant_documents(query)

if not retrieved_docs:
    response = "No relevant information found in the database."
else:
    context = " ".join([doc.page_content for doc in retrieved_docs[:1]])[:1000]
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer (only generate the final and precise solution, in one sentence):"
    response = query_huggingface_api(prompt)

print(response)

Context: hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  arrival_date_week_number  arrival_date_day_of_month  stays_in_weekend_nights  stays_in_week_nights  adults  children  babies      meal country market_segment distribution_channel  is_repeated_guest  previous_cancellations  previous_bookings_not_canceled reserved_room_type assigned_room_type  booking_changes deposit_type  agent  company  days_in_waiting_list   customer_type     adr  required_car_parking_spaces  total_of_special_requests reservation_status reservation_status_date
Resort Hotel            0        342               2015               July                        27                          1                        0                     0       2         0       0        BB     PRT         Direct               Direct                  0                       0                               0                  C                  C                3   No Deposit      0        0                     0    

In [21]:
# 🔹 Run Query 3
query = "Which month has the highest number of hotel bookings?"
retrieved_docs = retriever.get_relevant_documents(query)

if not retrieved_docs:
    response = "No relevant information found in the database."
else:
    context = " ".join([doc.page_content for doc in retrieved_docs[:1]])[:1000]
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer (only generate the final and precise solution, in one sentence):"
    response = query_huggingface_api(prompt)

print(response)

Context: hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  arrival_date_week_number  arrival_date_day_of_month  stays_in_weekend_nights  stays_in_week_nights  adults  children  babies      meal country market_segment distribution_channel  is_repeated_guest  previous_cancellations  previous_bookings_not_canceled reserved_room_type assigned_room_type  booking_changes deposit_type  agent  company  days_in_waiting_list   customer_type     adr  required_car_parking_spaces  total_of_special_requests reservation_status reservation_status_date
Resort Hotel            0        342               2015               July                        27                          1                        0                     0       2         0       0        BB     PRT         Direct               Direct                  0                       0                               0                  C                  C                3   No Deposit      0        0                     0    

In [31]:
# 🔹 Run Query 4
query = "What is the most common meal type selected by guests?"
retrieved_docs = retriever.get_relevant_documents(query)

if not retrieved_docs:
    response = "No relevant information found in the database."
else:
    context = " ".join([doc.page_content for doc in retrieved_docs[:3]])[:2000]
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer (only generate the final and precise solution, in one sentence):"
    response = query_huggingface_api(prompt)

print(response)

Context: hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  arrival_date_week_number  arrival_date_day_of_month  stays_in_weekend_nights  stays_in_week_nights  adults  children  babies      meal country market_segment distribution_channel  is_repeated_guest  previous_cancellations  previous_bookings_not_canceled reserved_room_type assigned_room_type  booking_changes deposit_type  agent  company  days_in_waiting_list   customer_type     adr  required_car_parking_spaces  total_of_special_requests reservation_status reservation_status_date
Resort Hotel            0        342               2015               July                        27                          1                        0                     0       2         0       0        BB     PRT         Direct               Direct                  0                       0                               0                  C                  C                3   No Deposit      0        0                     0    

In [39]:
# 🔹 Run Query 5
query = "What percentage of hotel bookings are canceled?"
retrieved_docs = retriever.get_relevant_documents(query)

if not retrieved_docs:
    response = "No relevant information found in the database."
else:
    context = " ".join([doc.page_content for doc in retrieved_docs[:10]])[:150000]
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer (only generate the final and precise solution, in one sentence):"
    response = query_huggingface_api(prompt)

print(response)

Context: hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  arrival_date_week_number  arrival_date_day_of_month  stays_in_weekend_nights  stays_in_week_nights  adults  children  babies      meal country market_segment distribution_channel  is_repeated_guest  previous_cancellations  previous_bookings_not_canceled reserved_room_type assigned_room_type  booking_changes deposit_type  agent  company  days_in_waiting_list   customer_type     adr  required_car_parking_spaces  total_of_special_requests reservation_status reservation_status_date
Resort Hotel            0        342               2015               July                        27                          1                        0                     0       2         0       0        BB     PRT         Direct               Direct                  0                       0                               0                  C                  C                3   No Deposit      0        0                     0    

# Hotel Bookings Data Analysis

## 1. Which country has the highest number of hotel bookings?
**Answer:** The country with the highest number of hotel bookings is **Portugal (PRT)**.

## 2. What is the most common market segment for hotel bookings?
**Answer:** The most common market segment for hotel bookings is **Online TA (Online Travel Agencies)**.

## 3. Which month has the highest number of hotel bookings?
**Answer:** The month with the highest number of hotel bookings is **August**.

## 4. What is the most common meal type selected by guests?
**Answer:** The most common meal type selected by guests is **Bed & Breakfast (BB)**.

## 5. What percentage of hotel bookings are canceled?
**Answer:** Approximately **37%** of hotel bookings are canceled.
